# Installs & tokens

In [19]:
%%capture
try:
    import mlflow
except ImportError:
    !pip install mlflow

In [20]:
%%capture
try:
    import dotenv
except ImportError:
    !pip install python-dotenv

In [21]:
# Log into huggingface via Kaggle Secrets or .env

import os
from dotenv import load_dotenv
import huggingface_hub

try:
    from kaggle_secrets import UserSecretsClient

    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
except ModuleNotFoundError:
    print("Not Kaggle environment. Skipping Kaggle secrets.")
    print("Trying to load HF_TOKEN from .env.")
    load_dotenv()
    HF_TOKEN = os.getenv("HF_TOKEN")
    print("Success!")

huggingface_hub.login(token=HF_TOKEN)

Not Kaggle environment. Skipping Kaggle secrets.
Trying to load HF_TOKEN from .env.
Success!


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Choose notebook parameters

In [22]:
import torch

## CHOOSE MODEL PARAMETERS #################################################

MODEL_NAME_POSTFIX='splitting-by-query'
NAME_MODEL_NAME = 'cointegrated/rubert-tiny' # 'DeepPavlov/distilrubert-tiny-cased-conversational-v1'
DESCRIPTION_MODEL_NAME = 'cointegrated/rubert-tiny'

DATA_PATH = 'data/'
RESULTS_DIR = 'train_results/'

# BATCH_SIZE=60 # uses 14.5GiB of 1 GPU
# NUM_WORKERS=2 # TODO: use multiple GPU, tune number of workers
# NUM_DEBUG_SAMPLES=None
# EPOCHS=10 # epochs > 8 => overfit; NOTE: can train for longer since we take best validation checkpoint anyway

BATCH_SIZE=1
NUM_WORKERS=0
NUM_DEBUG_SAMPLES=2
EPOCHS=2

PRELOAD_MODEL_NAME = 'cc12m_rubert_tiny_ep_1.pt' # preload ruclip
# PRELOAD_MODEL_NAME = None

POS_WEIGHT = 4.0 # TODO: infer from data

# USE_ALL_TRAIN_PAIRS = False
# MAX_SAMPLES_PER_EPOCH = None

USE_ALL_TRAIN_PAIRS = True
MAX_SAMPLES_PER_EPOCH = 2_500
# MAX_SAMPLES_PER_EPOCH = 2_500 * 12

DROPOUT = 0.5
# DROPOUT = None

# BEST_CKPT_METRIC = 'f1'
BEST_CKPT_METRIC = 'pos_acc'

VALIDATION_SPLIT=.05
TEST_SPLIT=.1
RANDOM_SEED=42
LR=9e-5
MOMENTUM=0.9
WEIGHT_DECAY=1e-2
CONTRASTIVE_MARGIN=1.5
CONTRASTIVE_THRESHOLD=0.3
SHEDULER_PATIENCE=3 # in epochs

DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

In [23]:
## CHOOSE DATA #########################################################

DATA_PATH=  'data/'

# --- Load source_df and pairwise_mapping_df ---
SOURCE_TABLE_NAME = 'tables_OZ_geo_5500/processed/OZ_geo_5500.csv'
PAIRWISE_TABLE_NAME = 'tables_OZ_geo_5500/processed/regex-pairwise-groups_num-queries=2_patterns-dict-hash=6dbf9b3ef9568e60cd959f87be7e3b26.csv'
IMG_DATASET_NAME = 'images_OZ_geo_5500'

In [24]:
## LOGGING PARAMS ######################################################################

# MLFLOW_URI = "http://176.56.185.96:5000"
# MLFLOW_URI = "http://localhost:5000"
MLFLOW_URI = None

MLFLOW_EXPERIMENT = "siamese/1fold"

TELEGRAM_TOKEN = None
# TELEGRAM_TOKEN = '' # set token to get notifications

# Definitions

In [25]:
# Imports
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import mlflow
from mlflow.models import infer_signature

from timm import create_model
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
from torch import optim, Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchinfo import summary
# import transformers
# from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer,\
#         get_linear_schedule_with_warmup
from transformers import AutoModel, AutoTokenizer

import cv2

from PIL import Image
from tqdm.auto import tqdm

# import json
# from itertools import product

# import datasets
# from datasets import Dataset, concatenate_datasets
# import argparse
import requests

# from io import BytesIO
# from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, f1_score
import matplotlib.pyplot as plt
from IPython import display
# import more_itertools

from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import f1_score
import mlflow
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import tempfile

In [26]:
def make_tg_report(text, token=None) -> None:
    method = 'sendMessage'
    chat_id = 324956476
    _ = requests.post(
            url='https://api.telegram.org/bot{0}/{1}'.format(token, method),
            data={'chat_id': chat_id, 'text': text} 
        ).json()

In [27]:
class RuCLIPtiny(nn.Module):
    def __init__(self, name_model_name):
        super().__init__()
        self.visual = create_model('convnext_tiny',
                                   pretrained=False, # TODO: берём претрейн
                                   num_classes=0,
                                   in_chans=3)  # out 768

        self.transformer = AutoModel.from_pretrained(name_model_name)
        name_model_output_shape = self.transformer.config.hidden_size  # dynamically get hidden size
        self.final_ln = torch.nn.Linear(name_model_output_shape, 768)  # now uses the transformer hidden size
        self.logit_scale = torch.nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

    @property
    def dtype(self):
        return self.visual.stem[0].weight.dtype

    def encode_image(self, image):
        return self.visual(image.type(self.dtype))

    def encode_text(self, input_ids, attention_mask):
        x = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        x = x.last_hidden_state[:, 0, :]
        x = self.final_ln(x)
        return x

    def forward(self, image, input_ids, attention_mask):
        image_features = self.encode_image(image)
        text_features = self.encode_text(input_ids, attention_mask)

        # normalized features
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        # cosine similarity as logits
        logit_scale = self.logit_scale.exp()
        logits_per_image = logit_scale * image_features @ text_features.t()
        logits_per_text = logits_per_image.t()

        return logits_per_image, logits_per_text

In [28]:
def get_transform():
    return transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        _convert_image_to_rgb,
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]), ])

def _convert_image_to_rgb(image):
    return image.convert("RGB")

class Tokenizers:
    def __init__(self):
        self.name_tokenizer = AutoTokenizer.from_pretrained(NAME_MODEL_NAME)
        self.desc_tokenizer = AutoTokenizer.from_pretrained(DESCRIPTION_MODEL_NAME)

    def tokenize_name(self, texts, max_len=77):
        tokenized = self.name_tokenizer.batch_encode_plus(texts,
                                                     truncation=True,
                                                     add_special_tokens=True,
                                                     max_length=max_len,
                                                     padding='max_length',
                                                     return_attention_mask=True,
                                                     return_tensors='pt')
        return torch.stack([tokenized["input_ids"], tokenized["attention_mask"]])
    
    def tokenize_description(self, texts, max_len=77):
        tokenized = self.desc_tokenizer(texts,
                                        truncation=True,
                                        add_special_tokens=True,
                                        max_length=max_len,
                                        padding='max_length',
                                        return_attention_mask=True,
                                        return_tensors='pt')
        return torch.stack([tokenized["input_ids"], tokenized["attention_mask"]])

class SiameseRuCLIPDataset(torch.utils.data.Dataset):
    def __init__(self, df=None, labels=None, df_path=None, images_dir=DATA_PATH+'images/'):
        # loads data either from path using `df_path` or directly from `df` argument
        self.df = pd.read_csv(df_path) if df_path is not None else df
        self.labels = labels
        self.images_dir = images_dir
        self.tokenizers = Tokenizers()
        self.transform = get_transform()
        # 
        self.max_len = 77
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        name_tokens = self.tokenizers.tokenize_name([str(row.name_first), 
                                               str(row.name_second)], max_len=self.max_len)
        name_first = name_tokens[:, 0, :] # [input_ids, attention_mask]
        name_second = name_tokens[:, 1, :]
        desc_tokens = self.tokenizers.tokenize_description([str(row.description_first), 
                                               str(row.description_second)])
        desc_first = desc_tokens[:, 0, :] # [input_ids, attention_mask]
        desc_second = desc_tokens[:, 1, :]
        im_first = cv2.imread(os.path.join(self.images_dir, row.image_name_first))
        im_first = cv2.cvtColor(im_first, cv2.COLOR_BGR2RGB)
        im_first = Image.fromarray(im_first)
        im_first = self.transform(im_first)
        im_second = cv2.imread(os.path.join(self.images_dir, row.image_name_second))
        im_second = cv2.cvtColor(im_second, cv2.COLOR_BGR2RGB)
        im_second = Image.fromarray(im_second)
        im_second = self.transform(im_second)
        label = self.labels[idx]
        return im_first, name_first, desc_first, im_second, name_second, desc_second, label

    def __len__(self,):
        return len(self.df)

In [29]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

class SiameseRuCLIP(nn.Module):
    def __init__(self,
                 device: str,
                 name_model_name: str,
                 description_model_name: str,
                 preload_model_name: str = None,
                 models_dir: str = None,
                 dropout: float = None):
        """
        Initializes the SiameseRuCLIP model.
        Required parameters:
          - models_dir: directory containing saved checkpoints.
          - name_model_name: model name for text (name) branch.
          - description_model_name: model name for description branch.
        """
        super().__init__()
        device = torch.device(device)

        # Initialize RuCLIPtiny
        self.ruclip = RuCLIPtiny(name_model_name)
        if preload_model_name is not None:
            std = torch.load(
                os.path.join(models_dir, preload_model_name),
                weights_only=True,
                map_location=device
            )
            self.ruclip.load_state_dict(std)
            self.ruclip.eval()
        self.ruclip = self.ruclip.to(device)

        # Initialize the description transformer
        self.description_transformer = AutoModel.from_pretrained(description_model_name)
        self.description_transformer = self.description_transformer.to(device)

        # Determine dimensionality
        vision_dim = self.ruclip.visual.num_features
        name_dim = self.ruclip.final_ln.out_features
        desc_dim = self.description_transformer.config.hidden_size
        self.hidden_dim = vision_dim + name_dim + desc_dim
        self.dropout = dropout

        # Define MLP head with optional dropout
        layers = [
            nn.Linear(self.hidden_dim, self.hidden_dim // 2),
            nn.ReLU(),
            *( [nn.Dropout(self.dropout)] if self.dropout is not None else [] ),
            nn.Linear(self.hidden_dim // 2, self.hidden_dim // 4),
        ]
        self.head = nn.Sequential(*layers).to(device)


    def encode_image(self, image):
        return self.ruclip.encode_image(image)

    def encode_name(self, name):
        return self.ruclip.encode_text(name[:, 0, :], name[:, 1, :])

    def encode_description(self, desc):
        last_hidden_states = self.description_transformer(desc[:, 0, :], desc[:, 1, :]).last_hidden_state
        attention_mask = desc[:, 1, :]
        return average_pool(last_hidden_states, attention_mask)

    def get_final_embedding(self, im, name, desc):
        image_emb = self.encode_image(im)
        name_emb = self.encode_name(name)
        desc_emb = self.encode_description(desc)

        # Concatenate the embeddings and forward through the head
        combined_emb = torch.cat([image_emb, name_emb, desc_emb], dim=1)
        final_embedding = self.head(combined_emb)
        return final_embedding

    def forward(self, im1, name1, desc1, im2, name2, desc2):
        out1 = self.get_final_embedding(im1, name1, desc1)
        out2 = self.get_final_embedding(im2, name2, desc2)
        return out1, out2

In [30]:
# # old
# class ContrastiveLoss(torch.nn.Module):
#     def __init__(self, margin=2.0):
#         super(ContrastiveLoss, self).__init__()
#         self.margin = margin
        
#     def __name__(self,):
#         return 'ContrastiveLoss'

#     def forward(self, output1, output2, label):
#         euclidean_distance = F.pairwise_distance(output1, output2)
#         pos = (1-label) * torch.pow(euclidean_distance, 2)
#         neg = label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
#         loss_contrastive = torch.mean( pos + neg )
#         return loss_contrastive

In [31]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin: float = 1.5, pos_weight: float = 4.0):
        super().__init__()
        self.margin      = margin
        self.pos_weight  = pos_weight

    def forward(self, output1, output2, label):
        d   = F.pairwise_distance(output1, output2)
        pos = (1 - label) * d.pow(2)                            # duplicates (label==0)
        neg = label * F.relu(self.margin - d).pow(2)            # different (label==1)
        return (self.pos_weight * pos + neg).mean()

In [32]:
# TODO: plot epoch after each train epoch in `train()`

from pathlib import Path

def plot_epoch(loss_history, filename="data/runs_artifacts/epoch_loss.png") -> None:
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    display.clear_output(wait=True)
    plt.figure(figsize=(6, 4))
    plt.title("Training loss")
    plt.xlabel("Iteration number")
    plt.ylabel("Loss")
    plt.plot(loss_history, 'b')
    plt.tight_layout()
    plt.savefig(filename)  # Save the plot to a file
    plt.show()

In [33]:
def evaluate_pair(output1, output2, target, threshold):
    euclidean_distance = F.pairwise_distance(output1, output2)
    # меньше границы, там где будет True — конкуренты
    cond = euclidean_distance < threshold
    pos_sum = 0
    neg_sum = 0
    pos_acc = 0
    neg_acc = 0

    for i in range(len(cond)):
        # 1 значит не конкуренты
        if target[i]:
            neg_sum+=1
            # 0 в cond значит дальше друг от друга чем threshold
            if not cond[i]:
                neg_acc+=1
        elif not target[i]:
            pos_sum+=1
            if cond[i]:
                pos_acc+=1

    return pos_acc, pos_sum, neg_acc, neg_sum

def predict(out1, out2, threshold=CONTRASTIVE_THRESHOLD):
    # вернёт 1 если похожи
    return F.pairwise_distance(out1, out2) < threshold

In [34]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
import mlflow
from copy import deepcopy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pathlib import Path

def validation(model,
               criterion,
               data_loader,
               epoch,
               device='cpu',
               split_name='validation',
               threshold=None,
               margin=1.5,
               steps=200,
               metric='f1'):
    """
    Runs one pass over `data_loader`, returning:
      pos_acc, neg_acc, avg_acc, f1, avg_loss, best_thr

    threshold sweep: if threshold is None, tests `steps` values in [0, margin]
    and picks the one that maximises either:
      - F1              if metric=='f1'
      - positive accuracy if metric=='pos_acc'
    """
    assert metric in ('f1', 'pos_acc'), "metric must be 'f1' or 'pos_acc'"

    model.eval()
    total_loss = 0.0
    all_d, all_lbl = [], []

    with torch.no_grad():
        for batch in data_loader:
            im1, n1, d1, im2, n2, d2, lbl = [t.to(device) for t in batch]
            out1, out2 = model(im1, n1, d1, im2, n2, d2)
            total_loss += criterion(out1, out2, lbl).item()
            all_d.append(F.pairwise_distance(out1, out2).cpu())
            all_lbl.append(lbl.cpu())

    distances = torch.cat(all_d)
    labels    = torch.cat(all_lbl)               # 0 = duplicate (positive), 1 = different (negative)
    avg_loss  = total_loss / len(data_loader)

    # === threshold sweep ===
    if threshold is None:
        grid = np.linspace(0.0, margin, steps)
        best_val, best_thr = -1.0, 0.0
        y_true = (labels.numpy() == 0).astype(int)   # 1 = positive
        for t in grid:
            y_pred = (distances.numpy() < t).astype(int)
            if metric == 'f1':
                val = f1_score(y_true, y_pred, zero_division=0)
            else:  # metric == 'pos_acc'
                # positive accuracy = TP / P
                pos_mask = (y_true == 1)
                val = (y_pred[pos_mask] == 1).mean() if pos_mask.sum() > 0 else 0.0
            if val > best_val:
                best_val, best_thr = val, t
        threshold = best_thr
    else:
        best_thr = threshold

    # === final metrics at chosen threshold ===
    preds    = (distances < threshold).long()
    pos_mask = (labels == 0)
    neg_mask = (labels == 1)

    pos_acc = (preds[pos_mask] == 1).float().mean().item() if pos_mask.any() else 0.0
    neg_acc = (preds[neg_mask] == 0).float().mean().item() if neg_mask.any() else 0.0
    avg_acc = (pos_acc + neg_acc) / 2.0
    f1      = f1_score((labels.numpy() == 0).astype(int),
                       preds.numpy(), zero_division=0)

    # log to Telegram / console
    report = (f"[{split_name}] Epoch {epoch} – "
              f"loss: {avg_loss:.4f}, "
              f"P Acc: {pos_acc:.3f}, "
              f"N Acc: {neg_acc:.3f}, "
              f"Avg Acc: {avg_acc:.3f}, "
              f"F1: {f1:.3f}, "
              f"thr*: {threshold:.3f} "
              f"(optimised: {metric})")
    print(report)
    make_tg_report(report, TELEGRAM_TOKEN)

    # log to MLflow under the chosen metric
    if MLFLOW_URI and split_name == 'validation':
        if metric == 'f1':
            mlflow.log_metric("valid_f1_score", f1, step=epoch)
        else:
            mlflow.log_metric("valid_pos_accuracy", pos_acc, step=epoch)

    return pos_acc, neg_acc, avg_acc, f1, avg_loss, threshold


In [35]:
from time import perf_counter
from datetime import timedelta

def train(model,
          optimizer,
          criterion,
          epochs_num,
          train_loader,
          valid_loader=None,
          device='cpu',
          print_epoch=False,
          models_dir=None,
          metric='f1'):
    """
    Trains for `epochs_num` epochs, using `validation(..., metric=metric)` each epoch.
    Uses the same `metric` to step the LR scheduler and to pick the best checkpoint.

    Returns:
      train_losses, val_losses, best_valid_metric, best_weights, thr_history
    """
    assert metric in ('f1', 'pos_acc'), "metric must be 'f1' or 'pos_acc'"

    model.to(device)
    train_losses, val_losses, thr_history = [], [], []
    best_valid_metric, best_threshold = float('-inf'), None
    best_weights = None

    scheduler = ReduceLROnPlateau(
        optimizer,
        mode="max",
        factor=0.1,
        patience=SHEDULER_PATIENCE,
        threshold=1e-4,
        threshold_mode='rel'
    )

    if models_dir:
        Path(models_dir).mkdir(parents=True, exist_ok=True)

    for epoch in range(1, epochs_num + 1):
        # ---- training ----
        model.train()
        total_train_loss = 0.0
        for batch in train_loader:
            im1, n1, d1, im2, n2, d2, lbl = [t.to(device) for t in batch]
            optimizer.zero_grad()
            out1, out2 = model(im1, n1, d1, im2, n2, d2)
            loss = criterion(out1, out2, lbl)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        train_losses.append(total_train_loss / len(train_loader))

        # ---- validation & checkpointing ----
        if print_epoch and valid_loader is not None:
            pos_acc, neg_acc, avg_acc, f1_val, val_loss, val_thr = validation(
                model,
                criterion,
                valid_loader,
                epoch,
                device=device,
                split_name='validation',
                threshold=None,
                margin=CONTRASTIVE_MARGIN,
                steps=200,
                metric=metric
            )
            val_losses.append(val_loss)
            thr_history.append(val_thr)

            # pick the metric value to step & compare
            cur_metric = pos_acc if metric == 'pos_acc' else f1_val
            scheduler.step(cur_metric)

            # save checkpoint every epoch if requested
            if models_dir:
                torch.save(model.state_dict(),
                           Path(models_dir) / f"checkpoint_epoch_{epoch}.pt")

            # update best if improved
            if cur_metric > best_valid_metric:
                best_valid_metric = cur_metric
                best_threshold     = val_thr
                best_weights       = deepcopy(model.state_dict())

        print(f'Epoch {epoch} done.')

    print(f"Best validation {metric}: {best_valid_metric:.3f}  (thr={best_threshold:.3f})")
    return train_losses, val_losses, best_valid_metric, best_weights, thr_history

# Prepare data

## Download data from HF

In [36]:
# Download models' weights & text/image datasets

from huggingface_hub import snapshot_download
from pathlib import Path

REPO_ID = "INDEEPA/clip-siamese"
LOCAL_DIR = Path("data/train_results")
LOCAL_DIR.mkdir(parents=True, exist_ok=True)

snapshot_download(
    repo_id=REPO_ID,
    repo_type='dataset',
    local_dir='data',
    allow_patterns=[
        "train_results/cc12m*.pt",
        SOURCE_TABLE_NAME, PAIRWISE_TABLE_NAME,
        f"{IMG_DATASET_NAME}.zip"
    ],
)

# The following shell command was removed for script compatibility:
# !unzip -n -q data/{IMG_DATASET_NAME}.zip -d data/

# If you need to unzip in Python, use:
# import zipfile
# with zipfile.ZipFile(f"data/{IMG_DATASET_NAME}.zip", 'r') as zip_ref:
#     zip_ref.extractall("data/")


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

'/home/anton/marketplace/clip-siamese/data'

In [37]:
source_df = pd.read_csv(DATA_PATH + SOURCE_TABLE_NAME)
pairwise_mapping_df = pd.read_csv(DATA_PATH + PAIRWISE_TABLE_NAME)

# Compute embeddings for soft negatives

In [38]:
DO_COMPUTE_SOFT_NEG_EMB = True
# DO_COMPUTE_SOFT_NEG_EMB = False

EMBEDDING_MODEL_NAME = 'sergeyzh/LaBSE-ru-turbo'
EMB_BATCH_SIZE = 512 if torch.cuda.is_available() else 8

NUM_EMBS = 2
# NUM_EMBS = None

In [39]:
# --- Construct 'name_and_description' column for all SKUs in source_df ---
source_df['name_and_description'] = source_df['name'].fillna('') + '.\n' + source_df['description'].fillna('')
display.display(source_df[['sku', 'name_and_description']].head())

,sku,name_and_description
0,1871769771,"Карты МИРА и РОССИИ настенные политические,160..."
1,1679550303,Схема линий скоростного транспорта Москвы (Мет...
2,1200553001,"Политическая карта МИРА 160х109 см, Карта мира..."
3,922231521,"Политическая карта МИРА настенная, 100х70см, ш..."
4,922230517,"Политическая карта МИРА настенная, 160х102см, ..."


In [50]:
# --- Compute and save embeddings for all SKUs in source_df ---
if DO_COMPUTE_SOFT_NEG_EMB:
    from sentence_transformers import SentenceTransformer
    from tqdm import tqdm
    from pathlib import Path
    import torch

    all_skus_df = source_df.copy()
    if NUM_EMBS is not None:
        all_skus_df = all_skus_df.head(NUM_EMBS)

    model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)

    emb_table = all_skus_df[['sku', 'name_and_description']].copy().reset_index(drop=True)
    candidate_texts = emb_table['name_and_description'].astype(str).tolist()

    embeddings = model.encode(
        candidate_texts,
        batch_size=EMB_BATCH_SIZE,
        show_progress_bar=True
    )

    emb_table['name_desc_emb'] = [emb.tolist() if hasattr(emb, 'tolist') else emb for emb in embeddings]
    emb_table = emb_table[['sku', 'name_desc_emb']]
    display.display(emb_table.head())

    # Save to parquet
    file_dir = Path('embeddings/OZ_geo_5500')
    file_name = f"{Path(SOURCE_TABLE_NAME).stem}_name-and-description_embeddings_num-rows={len(emb_table)}.parquet"
    full_file_path = Path(DATA_PATH) / file_dir / file_name
    full_file_path.parent.mkdir(parents=True, exist_ok=True)

    emb_table.to_parquet(full_file_path, index=False)
    print(f"Saved embeddings to:\n{file_dir / file_name}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,sku,name_desc_emb
0,1871769771,"[-0.020089328289031982, -0.05487040802836418, ..."
1,1679550303,"[-0.004182410426437855, -0.040884267538785934,..."


Saved embeddings to:
embeddings/OZ_geo_5500/OZ_geo_5500_name-and-description_embeddings_num-rows=2.parquet


In [51]:
if DO_COMPUTE_SOFT_NEG_EMB:
    import os
    from dotenv import load_dotenv
    from huggingface_hub import HfApi, login

    # Load HF_TOKEN from .env
    load_dotenv()
    hf_token = os.getenv("HF_TOKEN")
    if not hf_token:
        raise ValueError("HF_TOKEN not found in .env file")

    # Log into HuggingFace
    login(token=hf_token)

    # Upload the folder
    api = HfApi()
    api.upload_folder(
        folder_path=DATA_PATH / file_dir,  # Path to the local directory
        path_in_repo=str(file_dir),
        repo_id="INDEEPA/clip-siamese",
        repo_type="dataset",
    )

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


OZ_geo_5500_name-and-description_embeddings_num-rows=2.parquet:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

# Cluster soft negatives

In [52]:
# Suggest the correct path to the embedding file based on the context and previous file saving logic
CHOSEN_EMBEDDING_FILE = 'embeddings/OZ_geo_5500/OZ_geo_5500_name-and-description_embeddings_num-rows=2.parquet'

In [53]:
from huggingface_hub import hf_hub_download
import pandas as pd

# Download the chosen embedding file from HuggingFace Hub to DATA_PATH
from pathlib import Path

downloaded_emb_file = hf_hub_download(
    repo_id="INDEEPA/clip-siamese",
    repo_type="dataset",
    filename=CHOSEN_EMBEDDING_FILE,
    local_dir=DATA_PATH,
)

print(f"Downloaded embedding file to:\n{downloaded_emb_file}")
emb_table = pd.read_parquet(downloaded_emb_file)
emb_table.head()

Downloaded embedding file to:
data/embeddings/OZ_geo_5500/OZ_geo_5500_name-and-description_embeddings_num-rows=2.parquet


,sku,name_desc_emb
0,1871769771,"[-0.020089328289031982, -0.05487040802836418, ..."
1,1679550303,"[-0.004182410426437855, -0.040884267538785934,..."


In [54]:
from sklearn.cluster import HDBSCAN
import numpy as np

# Prepare the embeddings as a numpy array
embeddings = np.stack(emb_table['name_desc_emb'].values)

# Run HDBSCAN clustering using sklearn's implementation
clusterer = HDBSCAN(min_cluster_size=2, metric='cosine')
cluster_labels = clusterer.fit_predict(embeddings)

# Add cluster labels to the emb_table and assign to cluster_emb_table
cluster_emb_table = emb_table.copy()
cluster_emb_table['cluster'] = cluster_labels

# Print cluster label counts
print("Cluster label counts:")
display.display(cluster_emb_table['cluster'].value_counts().to_frame())

Cluster label counts:


,count
cluster,
-1,2


# Make pairwise dataset

In [ ]:
import ast

# Parse stringified lists into actual lists
for col in ['sku_pos', 'sku_hard_neg', 'sku_soft_neg']:
    pairwise_mapping_df[col] = pairwise_mapping_df[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

# --- Explode to long format: each row is a (sku_query, sku_pos/hard_neg/soft_neg) pair ---
def explode_pairs(df, col, label):
    return pd.DataFrame({
        'sku_query': df['sku_query'].repeat(df[col].str.len()),
        'sku_candidate': [sku for sublist in df[col] for sku in sublist],
        'label': label
    })

pairs_pos = explode_pairs(pairwise_mapping_df, 'sku_pos', 1)
pairs_hard_neg = explode_pairs(pairwise_mapping_df, 'sku_hard_neg', 0)
pairs_soft_neg = explode_pairs(pairwise_mapping_df, 'sku_soft_neg', 0.5)

# Combine all pairs
regex_pairwise_df = pd.concat([pairs_pos, pairs_hard_neg, pairs_soft_neg], ignore_index=True)
print(f"Pairs: pos={len(pairs_pos)}, hard_neg={len(pairs_hard_neg)}, soft_neg={len(pairs_soft_neg)}, total={len(regex_pairwise_df)}")


Unique query sku: 2


/tmp/ipykernel_116370/1367893158.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  regex_pairwise_df = pd.read_csv(TABLE_DATASET_PATH)


In [ ]:
from typing import Tuple
import numpy as np
import pandas as pd

def split_pairwise(
    df: pd.DataFrame,
    test_size: float = 0.20,
    random_state: int | None = None,
    use_all_train_pairs: bool = False,        # ← NEW
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Leakage-free DEV / TEST split for a pair-wise SKU dataset.

    When `use_all_train_pairs=False` (default) the behaviour is identical to the
    previous implementation: a single `sku_substitute` is chosen and paired with
    every remaining positive / negative of the current group.

    When `use_all_train_pairs=True` **every** remaining positive becomes an
    anchor.  For each anchor we create:
        • (anchor, other_positive)   for all other positives in the group
        • (anchor, negative)         for every remaining negative in the group
    Self-pairs are skipped.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain the columns ['sku_query', 'sku_candidate', 'label'].
    test_size : float, optional
        Fraction of positives *and* negatives to move to TEST, by default 0.20.
    random_state : int | None, optional
        Seed for the random number generator.
    use_all_train_pairs : bool, optional
        False → original «single substitute» logic.
        True  → build full Cartesian train pairs.

    Returns
    -------
    dev_df, test_df : Tuple[pd.DataFrame, pd.DataFrame]
        The DEV (train/validation) and TEST splits.
    """
    print("Building pairwise dataset...")

    rng = np.random.default_rng(random_state)
    test_rows, dev_rows = [], []
    test_entities: set[str] = set()

    # ──────────────────────────────────────────────────────────────────────────
    for q_sku, grp in df.groupby("sku_query"):
        pos_idx = grp.index[grp.label == 1].tolist()
        neg_idx = grp.index[grp.label == 0].tolist()

        # ---- 1) sample TEST rows --------------------------------------------
        n_pos = int(np.ceil(test_size * len(pos_idx))) if pos_idx else 0
        n_neg = int(np.ceil(test_size * len(neg_idx))) if neg_idx else 0

        pos_test = rng.choice(pos_idx, size=n_pos, replace=False) if n_pos else []
        neg_test = rng.choice(neg_idx, size=n_neg, replace=False) if n_neg else []

        test_rows.extend(pos_test)
        test_rows.extend(neg_test)

        # track every entity that landed in TEST
        test_entities.add(q_sku)
        test_entities.update(df.loc[pos_test, "sku_candidate"])
        test_entities.update(df.loc[neg_test, "sku_candidate"])

        # ---- 2) build DEV rows ----------------------------------------------
        remain_pos = list(set(pos_idx) - set(pos_test))
        remain_neg = list(set(neg_idx) - set(neg_test))

        if not remain_pos:                     # nothing left to anchor on
            continue

        # (a) ORIGINAL behaviour: one substitute ------------------------------
        if not use_all_train_pairs:
            sub_idx  = int(rng.choice(remain_pos))
            sub_sku  = df.loc[sub_idx, "sku_candidate"]

            # pair substitute with (other positives + negatives)
            for idx in remain_pos:
                if idx == sub_idx:             # skip self-pair
                    continue
                row = df.loc[idx].copy()
                row["sku_query"] = sub_sku
                dev_rows.append(row)

            for idx in remain_neg:
                row = df.loc[idx].copy()
                row["sku_query"] = sub_sku
                dev_rows.append(row)

        # (b) NEW behaviour: full Cartesian ----------------------------------
        else:
            for anchor_idx in remain_pos:                      # every pos is anchor
                anchor_sku = df.loc[anchor_idx, "sku_candidate"]

                # anchor × (other positives)
                for idx in remain_pos:
                    if idx == anchor_idx:
                        continue
                    row = df.loc[idx].copy()
                    row["sku_query"] = anchor_sku
                    dev_rows.append(row)

                # anchor × (negatives)
                for idx in remain_neg:
                    row = df.loc[idx].copy()
                    row["sku_query"] = anchor_sku
                    dev_rows.append(row)

    # ──────────────────────────────────────────────────────────────────────────
    test_df = df.loc[test_rows].reset_index(drop=True)
    dev_df  = pd.DataFrame(dev_rows).reset_index(drop=True)

    test_df = (
        df.loc[test_rows]
          .drop_duplicates(subset=["sku_query", "sku_candidate", "label"])
          .reset_index(drop=True)
    )
    dev_df  = (
        pd.DataFrame(dev_rows)
          .drop_duplicates(subset=["sku_query", "sku_candidate", "label"])
          .reset_index(drop=True)
    )

    # final purge: any pair touching a TEST entity is removed from DEV
    mask = ~(dev_df["sku_query"].isin(test_entities) |
             dev_df["sku_candidate"].isin(test_entities))
    dev_df = dev_df[mask].reset_index(drop=True)

    print("Done.")
    return dev_df, test_df


In [197]:
# split into train/val/test

dev_df,  test_df  = split_pairwise(regex_pairwise_df,  test_size=TEST_SPLIT, random_state=42)
train_df, val_df  = split_pairwise(dev_df,   test_size=VALIDATION_SPLIT, random_state=42, use_all_train_pairs=USE_ALL_TRAIN_PAIRS)

print('Split sizes:', len(train_df), len(val_df), len(test_df))
print('Unique sku per split:', train_df.sku_query.nunique(), val_df.sku_query.nunique(), test_df.sku_query.nunique(),)

# Optionally: check label distribution
print('Label counts (train):', train_df.label.value_counts().to_dict())
print('Label counts (val):', val_df.label.value_counts().to_dict())
print('Label counts (test):', test_df.label.value_counts().to_dict())

Building pairwise dataset...
Done.
Building pairwise dataset...
Done.
Split sizes: 80601 175 644
Unique sku per split: 237 15 20


In [198]:
# Print positive/hard_negative pairs count per each split 

import pandas as pd

# collect your splits in a dict
splits = {
    'train': train_df,
    'val':   val_df,
    'test':  test_df,
}

# build the summary_df records
records = []
for name, df in splits.items():
    vc = df['label'].value_counts()
    records.append({
        'split':    name,
        'hard_negative': vc.get(0, 0),
        'positive': vc.get(1, 0),
        'total':    len(df),
    })

# create a DataFrame and set the split name as index
summary_df = (
    pd.DataFrame(records)
      .set_index('split')
      .astype(int)
)

print(summary_df)

       hard_negative  positive  total
split                                
train          69559     11042  80601
val              146        29    175
test             551        93    644


In [199]:
import pandas as pd
from collections import Counter

def sanity_checks(train: pd.DataFrame, val: pd.DataFrame, test: pd.DataFrame):
    """
    Verify that:
      1) Each query SKU appears in exactly one split
      2) No SKU (query or candidate) overlaps across splits
      3) No duplicate pairs across splits
      4) Each split has at least one positive and one hard_negative
    """
    # 1) Query-level disjointness
    q_train = set(train["sku_query"])
    q_val   = set(val  ["sku_query"])
    q_test  = set(test ["sku_query"])
    assert not (q_train & q_val),   f"Query SKU overlap train↔val: {q_train & q_val}"
    assert not (q_train & q_test),  f"Query SKU overlap train↔test: {q_train & q_test}"
    assert not (q_val   & q_test),  f"Query SKU overlap val↔test:   {q_val   & q_test}"
    
    # 2) Global SKU disjointness (query OR candidate)
    def all_skus(df):
        return set(df["sku_query"]) | set(df["sku_candidate"])
    s_train, s_val, s_test = all_skus(train), all_skus(val), all_skus(test)
    assert not (s_train & s_val),   f"SKU overlap train↔val: {s_train & s_val}"
    assert not (s_train & s_test),  f"SKU overlap train↔test: {s_train & s_test}"
    assert not (s_val   & s_test),  f"SKU overlap val↔test:   {s_val   & s_test}"
    
    # 3) Unique pairs
    all_pairs = pd.concat([train, val, test], ignore_index=True)
    dupes = all_pairs.duplicated(subset=["sku_query","sku_candidate","label"])
    assert not dupes.any(), f"Found {dupes.sum()} duplicate pairs across splits"
    
    # 4) Label coverage in each split
    for name, df in [("train", train), ("val", val), ("test", test)]:
        labels = df["label"].unique()
        assert set(labels) == {0,1}, f"{name} split has labels {labels}, expected {{0,1}}"
    
    print("✅ All sanity checks passed!")

sanity_checks(train_df, val_df, test_df)

✅ All sanity checks passed!


In [200]:
# rename for compatibility with torch.Dataset

def rename_cols(df):
    df = df.rename(columns={
        col: col.replace("_query", "_first").replace("_candidate", "_second")
        for col in df.columns
        if "_query" in col or "_candidate" in col
    })
    return df

train_df = rename_cols(train_df)
val_df = rename_cols(val_df)
test_df = rename_cols(test_df)

In [ ]:
# save for later usage

folder_name = f'test={TEST_SPLIT}_val={VALIDATION_SPLIT}'

# # TODO: different naming if need to evaluate on train split later
# if USE_ALL_TRAIN_PAIRS:
#     folder_name += '_use-all-train-pairs'
#     folder_name += f'_max-samples-per-epoch={MAX_SAMPLES_PER_EPOCH}'

dataset_name = Path(TABLE_DATASET_PATH).parts[1]

common_file_prefix = (
    Path(DATA_PATH) / dataset_name / 'processed' /
    'pairwise-splits' / folder_name
)
common_file_prefix.mkdir(parents=True, exist_ok=True)

train_df.to_csv(common_file_prefix / 'train.csv', index=False)
val_df.to_csv(common_file_prefix / 'val.csv', index=False)
test_df.to_csv(common_file_prefix / 'test.csv', index=False)

In [202]:
# take samples for each split to debug on smaller subset if necessary

def sample_split(df: pd.DataFrame, num_samples: int | None, random_state: int) -> pd.DataFrame:
    """
    If num_samples is set, take up to that many random rows;
    otherwise just shuffle the entire DataFrame.
    Always resets the index.
    """
    if num_samples is not None:
        n = min(num_samples, len(df))
        out = df.sample(n=n, random_state=random_state)
    else:
        out = df.sample(frac=1, random_state=random_state)
    return out.reset_index(drop=True)

actual_train_df = sample_split(train_df, NUM_DEBUG_SAMPLES, RANDOM_SEED)
actual_val_df   = sample_split(val_df,   NUM_DEBUG_SAMPLES, RANDOM_SEED)
actual_test_df  = sample_split(test_df,  NUM_DEBUG_SAMPLES, RANDOM_SEED)

In [203]:
# NOTE: very important; set inverse target for siamese net:
# label=0 (distance is minimal) -> new_label=1 (pair is similar)
# label=1 (distance is maximal) -> new_label=0 (pair is dissimilar)

actual_train_df["label"] = 1 - actual_train_df["label"]
actual_val_df["label"]   = 1 - actual_val_df["label"]
actual_test_df["label"]  = 1 - actual_test_df["label"]

# Run training

In [204]:
from sklearn.metrics import f1_score                 # ← new
import numpy as np
import torch.nn.functional as F

def best_threshold(distances: torch.Tensor,
                   labels:    torch.Tensor,
                   steps:     int = 200,
                   margin:    float = 1.5):
    """
    Sweep `steps` evenly-spaced thresholds between 0 and `margin`
    and return the one that maximises duplicate-class F1.
    Labels: 0 = duplicate (positive), 1 = different (negative).
    """
    d   = distances.detach().cpu().numpy()
    y   = labels.detach().cpu().numpy()
    thr = np.linspace(0.0, margin, steps)

    best_f1, best_thr = -1.0, 0.0
    for t in thr:
        y_pred = (d < t).astype(int)          # 1 = duplicate prediction
        f1     = f1_score(1 - y, y_pred)      # make 1 = positive for sklearn
        if f1 > best_f1:
            best_f1, best_thr = f1, t
    return best_thr, best_f1


In [175]:

from torch.utils.data import DataLoader, WeightedRandomSampler

def _run():
    images_dir = os.path.join(DATA_PATH, IMG_DATASET_NAME)

    # ---------- 1) build DataLoaders ----------
    splits  = {'train': actual_train_df,
               'validation': actual_val_df,
               'test': actual_test_df}
    loaders = {}

    for split_name, df in splits.items():
        labels = df["label"].values
        ds     = SiameseRuCLIPDataset(df.drop(columns="label"),
                                      labels,
                                      images_dir=images_dir)

        if split_name == "train":
            cls_cnt        = np.bincount(labels, minlength=2)
            cls_weights    = 1.0 / cls_cnt
            sample_weights = cls_weights[labels]
            total = len(sample_weights)
            max_n = MAX_SAMPLES_PER_EPOCH or total
            n_samples = min(total, max_n)

            sampler = WeightedRandomSampler(
                sample_weights,
                num_samples=n_samples,
                replacement=True
            )
            loaders[split_name] = DataLoader(
                ds,
                batch_size=BATCH_SIZE,
                sampler=sampler,
                num_workers=NUM_WORKERS
            )
        else:
            loaders[split_name] = DataLoader(
                ds,
                batch_size=BATCH_SIZE,
                shuffle=False,
                num_workers=NUM_WORKERS
            )

    train_loader = loaders['train']
    valid_loader = loaders['validation']
    test_loader  = loaders['test']

    # ---------- 2) model / optimiser ----------
    print("Loading model and optimizer…")
    model = SiameseRuCLIP(
        DEVICE, NAME_MODEL_NAME,
        DESCRIPTION_MODEL_NAME,
        PRELOAD_MODEL_NAME,
        DATA_PATH + RESULTS_DIR,
        dropout=DROPOUT
    ).to(DEVICE)

    criterion = ContrastiveLoss(
        margin=CONTRASTIVE_MARGIN,
        pos_weight=POS_WEIGHT
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY
    )
    print("Done.")

    # ---------- 3) training ----------
    with tempfile.TemporaryDirectory() as tmp_ckpt_dir:
        (train_losses, val_losses,
         best_metric_val, best_weights, thr_history) = train(
            model, optimizer, criterion,
            EPOCHS, train_loader, valid_loader,
            print_epoch=True, device=DEVICE,
            models_dir=tmp_ckpt_dir,
            metric=BEST_CKPT_METRIC
        )

    print(f"→ Best validation {BEST_CKPT_METRIC}: {best_metric_val:.3f}")

    # ---------- 4) loss curves ----------
    epochs_ax = list(range(1, len(train_losses) + 1))
    fig, ax = plt.subplots()
    ax.plot(epochs_ax, train_losses, label='Train Loss')
    ax.plot(epochs_ax, val_losses,   label='Val   Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Training & Validation Loss by Epoch')
    ax.legend()
    if MLFLOW_URI:
        mlflow.log_figure(fig, 'loss_by_epoch.png')
    display.clear_output(wait=True)
    display.display(fig)
    plt.close(fig)

    # ---------- 5) pick threshold for the *best* model ----------
    model.load_state_dict(best_weights)
    (_, _, _, _, _, best_thr) = validation(
        model, criterion, valid_loader,
        epoch='best', device=DEVICE,
        split_name='validation',
        threshold=None,
        metric=BEST_CKPT_METRIC
    )
    print(f"Chosen threshold from validation: {best_thr:.3f}")

    # ---------- 6) final TEST ----------
    (test_pos_acc, test_neg_acc,
     test_acc, test_f1,
     test_loss, _) = validation(
        model, criterion, test_loader,
        epoch='test', device=DEVICE,
        split_name='test',
        threshold=best_thr,
        metric=BEST_CKPT_METRIC
    )

    # pick out the right test-metric value
    test_metric = test_pos_acc if BEST_CKPT_METRIC == 'pos_acc' else test_f1
    print(f"Test {BEST_CKPT_METRIC}: {test_metric:.3f}")

    # ---------- 7) save checkpoint ----------
    filename = (
        f"siamese_contrastive_test-{BEST_CKPT_METRIC}={test_metric:.3f}"
        f"{'_' + MODEL_NAME_POSTFIX if MODEL_NAME_POSTFIX else ''}"
        f"{'_' + PRELOAD_MODEL_NAME  if PRELOAD_MODEL_NAME else ''}"
        f"_best-thr={best_thr:.3f}.pt"
    )
    final_path = Path(DATA_PATH + RESULTS_DIR) / filename
    final_path.parent.mkdir(parents=True, exist_ok=True)
    torch.save(best_weights, final_path)
    print(f"Saved best‐{BEST_CKPT_METRIC} checkpoint to {final_path}")

    if MLFLOW_URI:
        mlflow.log_metric("test_pos_accuracy", test_pos_acc)
        mlflow.log_metric("test_neg_accuracy", test_neg_acc)
        mlflow.log_metric("test_accuracy",     test_acc)
        mlflow.log_metric("test_f1_score",     test_f1)
        mlflow.end_run()

_run()

/tmp/ipykernel_3777/1435658876.py:20: RuntimeWarning: divide by zero encountered in divide
  cls_weights    = 1.0 / cls_cnt


Loading model and optimizer…
Done.


KeyboardInterrupt: 